## Obtain planetary and stellar parameters

In [1]:
import json
from pathlib import Path

import iris
import numpy as np

In [2]:
from aeolus.const import init_const

In [3]:
# Initialise universal constants
constants = init_const()
au = 149597870700.0  # standard astronomical unit (mean Earth-Sun distance) [m] (from SOCRATES)
sun_radius = 6.95508 * 1e8  # Sun radius [m] (from TEPCat)
jupiter_radius = 6.9911 * 1e7  # mean Jupiter radius [m] (from Wikipedia)
jupiter_mass = 1.89852 * 1e27  # Jupiter mass [kg] (from TEPCat)
gravitational_constant = 6.67428 * 1e-11  # gravitational constant [m3 kg-1 s-2] (from TEPCat)

# Put universal constants into a container and store it in .json
universal = [
    {"name": "au", "units": "m", "value": au},
    {"name": "sun_radius", "units": "m", "value": sun_radius},
    {"name": "jupiter_radius", "units": "m", "value": jupiter_radius},
    {"name": "jupiter_mass", "units": "kg", "value": jupiter_mass},
    {"name": "gravitational_constant", "units": "m3 kg-1 s-2", "value": gravitational_constant},
]
path_to_constants = Path.cwd() / "constants"
path_to_constants.mkdir(parents=True, exist_ok=True)
with open(path_to_constants / "universal.json", "w") as f:
    json.dump(universal, f)

In [4]:
# Functions to derive (or just check) planetary and stellar parameters
def calc_planet_mean_angular_motion(planet_orbital_period):
    """Calculate the mean angular motion of a planet in [radians day-1]."""
    planet_mean_angular_motion = 2 * np.pi / planet_orbital_period
    return planet_mean_angular_motion


def calc_planet_rotation_rate(planet_mean_angular_motion):
    """Calculate the rotation rate of a planet in [radians s-1]."""
    planet_rotation_rate = planet_mean_angular_motion / 86400
    return planet_rotation_rate


def calc_planet_surface_gravity(planet_mass, planet_radius):
    """Calculate the surface gravity of a planet in [m s-2]."""
    planet_surface_gravity = gravitational_constant * planet_mass / (planet_radius ** 2)
    return planet_surface_gravity


def calc_star_constant_at_1_au(star_effective_temperature, star_radius):
    """Calculate the stellar flux at 1 AU, also known as the stellar constant, in [W m-2]."""
    star_constant_at_1_au = (
        constants.stefan_boltzmann * star_effective_temperature ** 4 * (star_radius / au) ** 2
    )
    return star_constant_at_1_au


def calc_star_irradiance(star_constant_at_1_au, planet_semi_major_axis):
    """Calculate the stellar irradience, also known as the stellar flux at the top of the planet's atmosphere, in [W m-2]."""
    star_irradiance = star_constant_at_1_au * (1 / planet_semi_major_axis) ** 2
    return f"{star_irradiance.data:0.2e}"

To calculate the stellar irradience, also known as the stellar flux at the top of planet's atmosphere, we use:

$F_{1_{au}} = \frac{L}{4 \pi a_{1_{au}}^2}$ (1)

$F_{x_{au}} = \frac{L}{4 \pi a_{x_{au}}^2}$ (2)

where $L$ is the stellar luminosity [W], $F_{1_{au}}$ and $F_{x_{au}}$ is the stellar irradience at 1 au and x au [W m-2], respectively, and $a_{1_{au}}=1 au$ and $a_{x_{au}}=x au$, the planet's semi-major axis [au].

From (1) $\frac{L}{4 \pi} = F_{1_{au}} a_{1_{au}}^2$, which when substituted into (2) gives: $F_{x_{au}} = F_{1_{au}} a_{1_{au}}^2 \frac{1}{a_{x_{au}}^2} = F_{1_{au}} (\frac{a_{1_{au}}}{a_{x_{au}}})^2$.

In [5]:
# Planetary and stellar parameters from TEPCat and/or used by the UM
ASTRO_MASTER = {
    "hatp11b": {
        "planet_damp_height": {
            "mdh_0": iris.cube.Cube(4.6e6, units="m", long_name="damp_height"),
        },
        "planet_domain_height": {
            "mdh_0": iris.cube.Cube(4.6e6, units="m", long_name="height_domain"),
        },  # calc_astro_parameters_from_um.ipynb
        "planet_equilibrium_temperature": {"tepcat": iris.cube.Cube(838, units="K")},
        "planet_mass": {"tepcat": iris.cube.Cube(jupiter_mass * 0.0873, units="kg")},
        "planet_mean_angular_motion": {
            "tepcat": iris.cube.Cube(1.2854307093247925, units="radians day-1"),
            "um": iris.cube.Cube(1.2854307093247925, units="radians day-1", long_name="planet_dm"),
        },  # calc_planet_mean_angular_motion()
        "planet_mean_molar_mass_of_air": {
            "mdh_0": iris.cube.Cube(2.3659217693209884, units="g mol-1"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_orbital_period": {"tepcat": iris.cube.Cube(4.888, units="day")},
        "planet_radius": {
            "tepcat": iris.cube.Cube(jupiter_radius * 0.3890, units="m"),  # 27195379.0
            "um": iris.cube.Cube(27195379.0, units="m", long_name="planet_radius"),
        },
        "planet_rotation_rate": {
            "tepcat": iris.cube.Cube(1.4877670246814728e-05, units="radians s-1"),
            "um": iris.cube.Cube(1.4877670246814728e-05, units="radians s-1", long_name="omega"),
        },  # calc_planet_rotation_rate()
        "planet_semi_major_axis": {
            "tepcat": iris.cube.Cube(0.05259, units="1"),
            "um": iris.cube.Cube(0.05259, units="au", long_name="planet_a"),
        },
        "planet_specific_gas_constant": {
            "mdh_0": iris.cube.Cube(3514.2581245981864, units="J K-1 kg-1", long_name="r"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_specific_heat_capacity": {
            "mdh_0": iris.cube.Cube(12383.125443324243, units="J K-1 kg-1", long_name="cp"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_surface_gravity": {
            "tepcat": iris.cube.Cube(
                13.2, units="m s-2"
            ),  # calc_planet_surface_gravity()=14.956969613431541
            "um": iris.cube.Cube(13.2, units="m s-2", long_name="g"),
        },
        "star_constant_at_1_au": {
            "tepcat": iris.cube.Cube(298.48162311, units="W m-2"),
            "um": iris.cube.Cube(298.48162311, units="W m-2", long_name="sc"),
        },  # calc_star_constant_at_1_au()
        "star_effective_temperature": {"tepcat": iris.cube.Cube(4780, units="K")},
        "star_irradiance": {"um": iris.cube.Cube(1.08e05, units="W m-2")},  # calc_star_irradiance()
        "star_radius": {
            "tepcat": iris.cube.Cube(sun_radius * 0.683, units="m"),  # 475031964.00000006
            "um": iris.cube.Cube(475031964.00000006, units="m", long_name="stellar_radius"),
        },
    },
    "hd189733b": {
        "planet_damp_height": {"mdh_0": iris.cube.Cube(3.2e6, units="m", long_name="damp_height")},
        "planet_domain_height": {
            "mdh_0": iris.cube.Cube(3.2e6, units="m", long_name="height_domain")
        },  # calc_astro_parameters_from_um.ipynb
        "planet_equilibrium_temperature": {"tepcat": iris.cube.Cube(1191, units="K")},
        "planet_mass": {"tepcat": iris.cube.Cube(jupiter_mass * 1.150, units="kg")},
        "planet_mean_angular_motion": {
            "tepcat": iris.cube.Cube(2.831539119954748, units="radians day-1"),
            "um": iris.cube.Cube(2.831539119954748, units="radians day-1", long_name="planet_dm"),
        },  # calc_planet_mean_angular_motion()
        "planet_mean_molar_mass_of_air": {
            "mdh_0": iris.cube.Cube(2.364688445909212, units="g mol-1"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_orbital_period": {"tepcat": iris.cube.Cube(2.219, units="day")},
        "planet_radius": {
            "tepcat": iris.cube.Cube(jupiter_radius * 1.151, units="m"),  # 80467561.0
            "um": iris.cube.Cube(80467561.0, units="m", long_name="planet_radius"),
        },
        "planet_rotation_rate": {
            "tepcat": iris.cube.Cube(3.277244351799477e-05, units="radians s-1"),
            "um": iris.cube.Cube(3.277244351799477e-05, units="radians s-1", long_name="omega"),
        },  # calc_planet_rotation_rate()
        "planet_semi_major_axis": {
            "tepcat": iris.cube.Cube(0.03142, units="1"),
            "um": iris.cube.Cube(3.142e-2, units="au", long_name="planet_a"),
        },
        "planet_specific_gas_constant": {
            "mdh_0": iris.cube.Cube(3516.0910158729716, units="J K-1 kg-1", long_name="r"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_specific_heat_capacity": {
            "mdh_0": iris.cube.Cube(1.25e4, units="J K-1 kg-1", long_name="cp"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_surface_gravity": {
            "tepcat": iris.cube.Cube(
                21.5, units="m s-2"
            ),  # calc_planet_surface_gravity()=22.50483138506895
            "um": iris.cube.Cube(21.5, units="m s-2", long_name="g"),
        },
        "star_constant_at_1_au": {
            "tepcat": iris.cube.Cube(450.78120227, units="W m-2"),
            "um": iris.cube.Cube(454.36616584739005, units="W m-2", long_name="sc"),
        },  # calc_star_constant_at_1_au()
        "star_effective_temperature": {"tepcat": iris.cube.Cube(5050, units="K")},
        "star_irradiance": {"um": iris.cube.Cube(4.60e05, units="W m-2")},  # calc_star_irradiance()
        "star_radius": {
            "tepcat": iris.cube.Cube(sun_radius * 0.752, units="m"),  # 523022016.0
            "um": iris.cube.Cube(523022016.0, units="m", long_name="stellar_radius"),
        },
    },
    "hd209458b": {
        "planet_damp_height": {"mdh_0": iris.cube.Cube(9.0e6, units="m", long_name="damp_height")},
        "planet_domain_height": {
            "mdh_0": iris.cube.Cube(9.0e6, units="m", long_name="height_domain")
        },  # calc_astro_parameters_from_um.ipynb
        "planet_equilibrium_temperature": {"tepcat": iris.cube.Cube(1459, units="K")},
        "planet_mass": {"tepcat": iris.cube.Cube(jupiter_mass * 0.714, units="kg")},
        "planet_mean_angular_motion": {
            "tepcat": iris.cube.Cube(1.782463916930379, units="radians day-1"),
            "um": iris.cube.Cube(1.782463916930379, units="radians day-1", long_name="planet_dm"),
        },  # calc_planet_mean_angular_motion()
        "planet_mean_molar_mass_of_air": {
            "mdh_0": iris.cube.Cube(2.3643128599845658, units="g mol-1"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_orbital_period": {"tepcat": iris.cube.Cube(3.525, units="day")},
        "planet_radius": {
            "tepcat": iris.cube.Cube(jupiter_radius * 1.380, units="m"),  # 96477179.99999999
            "um": iris.cube.Cube(9.0e7, units="m", long_name="planet_radius"),
        },
        "planet_rotation_rate": {
            "tepcat": iris.cube.Cube(2.0630369408916424e-05, units="radians s-1"),
            "um": iris.cube.Cube(2.0630369408916424e-05, units="radians s-1", long_name="omega"),
        },  # calc_planet_rotation_rate()
        "planet_semi_major_axis": {
            "tepcat": iris.cube.Cube(0.04747, units="1"),
            "um": iris.cube.Cube(4.747e-2, units="au", long_name="planet_a"),
        },
        "planet_specific_gas_constant": {
            "mdh_0": iris.cube.Cube(3516.649568980594, units="J K-1 kg-1", long_name="r"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_specific_heat_capacity": {
            "mdh_0": iris.cube.Cube(1.28e4, units="J K-1 kg-1", long_name="cp"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_surface_gravity": {
            "tepcat": iris.cube.Cube(
                9.3, units="m s-2"
            ),  # calc_planet_surface_gravity()=11.169475805973333
            "um": iris.cube.Cube(9.3, units="m s-2", long_name="g"),
        },
        "star_constant_at_1_au": {
            "tepcat": iris.cube.Cube(2317.02533209, units="W m-2"),
            "um": iris.cube.Cube(2335.4521240173804, units="W m-2", long_name="sc"),
        },  # calc_star_constant_at_1_au()
        "star_effective_temperature": {"tepcat": iris.cube.Cube(6117, units="K")},
        "star_irradiance": {"um": iris.cube.Cube(1.04e06, units="W m-2")},  # calc_star_irradiance()
        "star_radius": {
            "tepcat": iris.cube.Cube(sun_radius * 1.162, units="m"),  # 808180296.0
            "um": iris.cube.Cube(8.08e08, units="m", long_name="stellar_radius"),
        },
    },
    "wasp17b": {
        "planet_damp_height": {"mdh_0": iris.cube.Cube(3.7e7, units="m", long_name="damp_height")},
        "planet_domain_height": {
            "mdh_0": iris.cube.Cube(3.7e7, units="m", long_name="height_domain")
        },  # calc_astro_parameters_from_um.ipynb
        "planet_equilibrium_temperature": {"tepcat": iris.cube.Cube(1755, units="K")},
        "planet_mass": {"tepcat": iris.cube.Cube(jupiter_mass * 0.477, units="kg")},
        "planet_mean_angular_motion": {
            "tepcat": iris.cube.Cube(1.6822450621632092, units="radians day-1"),
            "um": iris.cube.Cube(1.6822450621632092, units="radians day-1", long_name="planet_dm"),
        },  # calc_planet_mean_angular_motion()
        "planet_mean_molar_mass_of_air": {
            "mdh_0": iris.cube.Cube(2.3629018165415685, units="g mol-1"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_orbital_period": {"tepcat": iris.cube.Cube(3.735, units="day")},
        "planet_radius": {
            "tepcat": iris.cube.Cube(jupiter_radius * 1.932, units="m"),  # 135068052.0
            "um": iris.cube.Cube(135068052.0, units="m", long_name="planet_radius"),
        },
        "planet_rotation_rate": {
            "tepcat": iris.cube.Cube(1.947042896022233e-05, units="radians s-1"),
            "um": iris.cube.Cube(1.947042896022233e-05, units="radians s-1", long_name="omega"),
        },  # calc_planet_rotation_rate()
        "planet_semi_major_axis": {
            "tepcat": iris.cube.Cube(0.05135, units="1"),
            "um": iris.cube.Cube(5.135e-2, units="au", long_name="planet_a"),
        },
        "planet_specific_gas_constant": {
            "mdh_0": iris.cube.Cube(3518.749590776207, units="J K-1 kg-1", long_name="r"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_specific_heat_capacity": {
            "mdh_0": iris.cube.Cube(1.32e4, units="J K-1 kg-1", long_name="cp"),
        },  # calc_astro_parameters_from_atmo.ipynb
        "planet_surface_gravity": {
            "tepcat": iris.cube.Cube(
                3.16, units="m s-2"
            ),  # calc_planet_surface_gravity()=3.31308596848587
            "um": iris.cube.Cube(3.31, units="m s-2", long_name="g"),
        },
        "star_constant_at_1_au": {
            "tepcat": iris.cube.Cube(5653.16463656, units="W m-2"),
            "um": iris.cube.Cube(5698.123009274174, units="W m-2", long_name="sc"),
        },  # calc_star_constant_at_1_au()
        "star_effective_temperature": {"tepcat": iris.cube.Cube(6550, units="K")},
        "star_irradiance": {"um": iris.cube.Cube(2.16e06, units="W m-2")},  # calc_star_irradiance()
        "star_radius": {
            "tepcat": iris.cube.Cube(sun_radius * 1.583, units="m"),  # 1100989164.0
            "um": iris.cube.Cube(1100989164.0, units="m", long_name="stellar_radius"),
        },
    },
}

In [6]:
# Select planetary and stellar parameters for running the UM
ASTRO_UM = {}
for planet, param_dicts in ASTRO_MASTER.items():
    ASTRO_UM[planet] = {}
    param_dict = {x: param_dicts[x] for x in param_dicts if x not in "tex"}
    for key, value in param_dict.items():
        if "mdh_0" in value.keys():
            ASTRO_UM[planet][key] = value["mdh_0"]  # add damp_height, domain_height, mu, Rs and cp
        else:
            if "um" not in value.keys():
                ASTRO_UM[planet][key] = value["tepcat"]  # add TEPCat values
            else:
                ASTRO_UM[planet][key] = value["um"]  # add UM values

# Put selected planetary and stellar parameters into a container
params_container = {}
for planet, param_dict in ASTRO_UM.items():
    params_container[planet] = []
    for key, value in param_dict.items():
        param = {"name": key, "units": str(value.units), "value": float(value.data)}
        params_container[planet].append(param)

# Store planetary-stellar parameters container by planet in .json
for planet, params in params_container.items():
    with open(path_to_constants / f"{planet}.json", "w") as f:
        json.dump(params, f)

For comparison with Moses2021* 

*Moses, J.I., Tremblin, P., Venot, O. et al. Chemical variation with altitude and longitude on exo-Neptunes: Predictions for Ariel phase-curve observations. Exp Astron (2021). https://doi.org/10.1007/s10686-021-09749-1

Find the rotation period and rotation rate of an exo-Neptune with $T_{eff}$=700 K using Moses2021 stellar and planetary parameters and the following formulas:

star mass $M_s=\frac{g_s{R_s}^2}{G}$, planet orbital period $T_{obr} = 2\pi\sqrt{\frac{{a}^3}{GM_s}}$, planet rotation rate $\omega=\frac{2\pi}{T_{orb}}$

In [7]:
log_g_s_in_cgs = 4.5  # (in cgs units)
R_s = 0.7 * sun_radius  # [m]
M_s = ((10 ** log_g_s_in_cgs) / 100 * R_s ** 2) / gravitational_constant  # [m]
planet_orbital_period = (
    2 * np.pi * np.sqrt(((0.067294 * au) ** 3) / (gravitational_constant * M_s))
)  # [s]
planet_rotation_rate = 2 * np.pi / planet_orbital_period  # [rad s-1]
print(planet_orbital_period / 86400, planet_rotation_rate)

8.484350272991739 8.571316580118899e-06
